In [12]:
from scipy.stats import t
from scipy.stats import norm;
import matplotlib.pyplot as plt
from PIL import Image
import random;
import numpy;
import math;
import scipy.integrate as integrate;
import scipy.special as special;

# Generate People

In [19]:
pathMeanTimes = [[],[],[],[],[]];
pathStdOfTimes = [[],[],[],[],[]];
pathDistances = [];
percentOfTimeInEachRegion = [[],[],[],[],[]];
P_I = [[],[],[],[],[]];
regionConcent = [120,110,80,200,230,100,150];

numberOfPeople = 2;

for personi in range(0,numberOfPeople):

    #1801 Fry Street Falcon Heights, MN 55113 to 163 Bridge Street, Saint Paul, MN 55126 
    randForPI=[]
    for randpi in range(0,5):
        randForPI.append(random.uniform(0,1));
        
    #via Snelling Ave and Highway 96W. #T = 16min. D = 8.9 mi
    #Follow Garden Ave to Snelling Ave
    #1 min (0.2 mi)
    #Turn left at the 2nd cross street onto Snelling Ave
    #8 min (4.9 mi)
    #Follow Hamline Ave N and Hwy 96 W to Bridge St in Shoreview
    #7 min (3.8 mi)
    pathMeanTimes[0].append(random.normalvariate(960,60));#Path 0 mean for this user time is random based on normal curve centered at 960 with a standard deviation of 60
    pathStdOfTimes[0].append(random.normalvariate(60,10));#Path 0 standard deviation is random based on normal curve centered at 60 with a standard deviation of 10
    pathDistances.append(8.9);
    P_I[0].append(randForPI[0]/sum(randForPI));#random probability for p_i divide by sum so that they add to 1
    percentOfTimeInEachRegion[0].append([.33,.12,.3,0,0,.25,0]);

    #via MN 36-E. T=15min. D=10.2 mi
    #Get on MN-36 E in Roseville from Snelling Ave
    #4 min (1.3 mi)
    #Continue on MN-36 E. Take Exit 113 to Rice St. Take exit 45 from I-694 W/US-10 W
    #8 min (6.7 mi)
    #Follow Rice St to Bridge St
    #4 min (2.2 mi)
    pathMeanTimes[1].append(random.normalvariate(900,45));
    pathStdOfTimes[1].append(random.normalvariate(45,5));
    pathDistances.append(10.2);
    P_I[1].append(randForPI[1]/sum(randForPI));
    percentOfTimeInEachRegion[1].append([0,.18,0,.07,.12,.32,.31]);

    #via Snelling Ave and County Rd F. T = 17min. D = 8.4 mi
    #Follow Garden Ave to Snelling Ave
    #1 min (0.2 mi)
    #Turn left at the 2nd cross street onto Snelling Ave
    #8 min (4.9 mi)
    #Follow County Rd F West and Snail Lake Blvd to Bridge St in Shoreview
    #8 min (3.3 mi)
    pathMeanTimes[2].append(random.normalvariate(1020,70));
    pathStdOfTimes[2].append(random.normalvariate(70,11));
    pathDistances.append(8.4);
    P_I[2].append(randForPI[2]/sum(randForPI));
    percentOfTimeInEachRegion[2].append([.23,.22,.3,0,0,.25,0]);

    #via Victorian T=19min. D = 8.6 mi
    #Follow Garden Ave to Snelling Ave
    #1 min (0.2 mi)
    #Take Victoria St N to Snail Lake Rd in Shoreview
    #16 min (7.6 mi)
    #Turn right onto Snail Lake Rd
    #2 min (0.6 mi)
    #Turn left onto Rice St
    #22 s (0.2 mi)
    #Turn left onto Bridge St
    #18 s (167 ft)
    pathMeanTimes[3].append(random.normalvariate(1140,75));
    pathStdOfTimes[3].append(random.normalvariate(75,9));
    pathDistances.append(8.6);
    P_I[3].append(randForPI[3]/sum(randForPI));
    percentOfTimeInEachRegion[3].append([.07,.22,.17,.19,0,.35,0]);

    #via MN 36 and Rice ST. T = 17.5 min D =8.9 mi
    #Follow Garden Ave to Snelling Ave
    #1 min (0.2 mi)
    #Take MN-36 E and Rice St to Bridge St
    #16 min (8.7 mi)
    #Turn left onto Bridge St
    #18 s (167 ft)
    pathMeanTimes[4].append(random.normalvariate(1050,65));
    pathStdOfTimes[4].append(random.normalvariate(65,8));
    pathDistances.append(8.9);
    P_I[4].append(randForPI[4]/sum(randForPI));
    percentOfTimeInEachRegion[4].append([0,.18,0,.07,.11,.33,.31]);

    #print("Path Mean Times in seconds: ");
    #print(pathMeanTimes);
    #print("\nPath Standard Deviation of Times in seconds: ");
    #print(pathStdOfTimes);
    #print("\nPath Distances in miles:");
    #print(pathDistances);
    #print("\nPath Percent in each Region: ");
    #print(percentOfTimeInEachRegion);
    #print("\nP_I(x):");
    #print(P_I);

In [24]:
#determining f(x,t)
def fexposure(path, time,concens):#path = path, time = time, concens = list of concentrations in each region
    #requires global specification of percent of time in each reagion, percentOfTimeInEachRegion
    #which is a list of lists.  There is one list for each path and that list is the list
    #of percents of times that path spends in each region.  This list must be of length len(concens)
    actualCITI = [];
    actualTimeInEachRegion = [];    
    for r in range(0,len(percentOfTimeInEachRegion[path])):
        actualTimeInEachRegion.append(percentOfTimeInEachRegion[path][0][r]*time);
    for regionNum in range(0,len(actualTimeInEachRegion)): #sum of atualCITI = f(x,t)
        actualCITI.append(concens[regionNum]*actualTimeInEachRegion[regionNum]);
    return sum(actualCITI);

In [25]:
#########
#THEORETICALLY MINIMAL MSE FOR CHANGING ENVIRONMENT BEGIN



low = 200
high = 2650
#These are the limits of integraton for time in the average value calculations for f_ave and for the MSE.
#They must be carefully set so that you integrate over all significantly likely times, but not
#so wide that you get division by 0 for times for which p(t) is indistinguishable from 0.

#########
#THEORETICALLY MINIMAL MSE START
def pt(t,pind,indPathMeans,indPathStds): #calculates true probability of time t for:
    #pind = list of individual true route selecction probabilities
    #indPathMeans = list of individual's mean travel times on each path
    #indPathStds = list of individual's standard deviations of travel times on each path
    total = 0;
    for i in range(0,len(pind)):
        total = total + norm.pdf(t,indPathMeans[i],indPathStds[i])*pind[i];
    return total



def avef(baseconcentrations,pind,indPathMeans,indPathStds): #average f_environ(x,t) over all routes 
    #times and environments
    #baseconcentrations = list of base pollutant concentrations in each cell
    #pind = list of individual true route selecction probabilities
    #indPathMeans = list of individual's mean travel times on each path
    #indPathStds = list of individual's standard deviations of travel times on each path
    total = 0;
    timeNormalization = integrate.quad(pt,low,high,args=(pind,indPathMeans,indPathStds))[0];
    for i in range(0,len(pind)):
        def f(t):
            return fexposure(i,t,(baseconcentrations))*pt(t,pind,indPathMeans,indPathStds);
        total = total + integrate.quad(f,low,high,args=())[0];
    return total/(len(pind)*timeNormalization);


def integrand(t, concens,pind,indPathMeans,indPathStds):#integrand in the MSE calculation
    #concens = list of pollutant concentrations in the cells
    #pind = list of individual true route selecction probabilities
    #indPathMeans = list of individual's mean travel times on each path
    #indPathStds = list of individual's standard deviations of travel times on each path
    term1 = 0;
    term2sum = 0;
    term2 = 0;
    for i in range(0,len(pathMeanTimes)):
        term1 = term1 + ((fexposure(i,t,concens))**2)*norm.pdf(t,indPathMeans[i],indPathStds[i])*pind[i];
    for i in range(0,len(pathMeanTimes)):
        term2sum = term2sum + fexposure(i,t,concens)*norm.pdf(t,indPathMeans[i],indPathStds[i])*pind[i];
    term2 = (term2sum**2)/pt(t,pind,indPathMeans,indPathStds);
    return term1 - term2;


def theoreticalmse(M,baseconc,confidencepct,pind,indPathMeans,indPathStds):#computes confidence interval
    #for relative MSE with:
    #M = number of runs (trials)
    #baseconc = base concentration list
    #confidencepct  = confidence level in percent
    #pind = list of individual true route selecction probabilities
    #indPathMeans = list of individual's mean travel times on each path
    #indPathStds = list of individual's standard deviations of travel times on each path
    sqrunErAve = 0.0; #running average mean squared error
    frunAve =  0.0; # running average average value of f_epsilon(x,t)
    sqErtotal = 0.0; # total running square error
    relErrorList = [];
    errorList = [];
    averagefValue = avef(baseconc,pind,indPathMeans,indPathStds);
    for trial in range(0,M):
        concens = [0,0,0,0,0,0,0];
        for region in range(len(baseconc)):
            concens[region] = baseconc[region]+random.uniform(-0.9,0.9);
        mse = integrate.quad(integrand,low,high,args=(concens,pind,indPathMeans,indPathStds))[0];
        #integrate from low to high for about 6 standard deviations above and below mean times
        errorList.append(mse);
        relErrorList.append(mse/(averagefValue**2));
        print(str(trial), end="");
    #
    std = numpy.std(relErrorList,ddof=1); #sample stdev of observed relative error
    tcv = t.ppf((100-confidencepct)/200,df = M-1,loc = 0, scale = 1); #critical value for our confidence
    mseest = numpy.mean(relErrorList);
    mseSqrtest = numpy.sqrt(mseest);
    confidence = [mseest - (-tcv*std)/numpy.sqrt(M), mseest +(-tcv*std)/numpy.sqrt(M)];
    #
    msePctest = mseest*100;
    mseSqrtPctest = mseSqrtest*100;
    msePctConfidence = [];
    mseSqrtPctConfidence = [];
    for i in range(0,len(confidence)):
        msePctConfidence.append(100*confidence[i]);
        mseSqrtPctConfidence.append(100*numpy.sqrt(confidence[i]));
    
    return [msePctest,msePctConfidence,mseSqrtPctest, mseSqrtPctConfidence,confidencepct];


#########
#THEORETICALLY MINIMAL MSE END







In [28]:
numberOfWeathersToGenerate = 14;
base=[1.1,1.8,1.4,0.9,1.6,1.0,1.8];
savedWeather = [];
savedTheoryMSE=[];


for weatheri in range(0,numberOfWeathersToGenerate):
    randWeather=[];
    for i in range(0,len(base)):
        randWeather.append( base[i] + random.uniform(-0.1,0.1))

    savedWeather.append(randWeather);
    
    for personi in range(0,numberOfPeople):
        P_IforPerson = [P_I[0][personi],P_I[1][personi],P_I[2][personi],P_I[3][personi],P_I[4][personi]];
        pathMeanTimesForPerson=[pathMeanTimes[0][personi],pathMeanTimes[1][personi],pathMeanTimes[2][personi],pathMeanTimes[3][personi],pathMeanTimes[4][personi]]
        pathStdOfTimesForPerson=[pathStdOfTimes[0][personi],pathStdOfTimes[1][personi],pathStdOfTimes[2][personi],pathStdOfTimes[3][personi],pathStdOfTimes[4][personi]]

        theorymse = theoreticalmse(10,base,99,P_IforPerson,pathMeanTimesForPerson,pathStdOfTimesForPerson);
        print("\n");
        print(str(theorymse[4])+" percent confidence intervals.");
        print("mse percent = "+str(theorymse[0])+"  confidence int: "+str(theorymse[1]));
        print("sqrt of mse percent = "+str(theorymse[2])+"  confidence int: "+str(theorymse[3]));
        print(str(base));

[0.19277590395804933, 0.0684060271041379, 0.26991207307553794, 0.1886600419298712, 0.2802459539324036]
[1114.017641967696, 903.5775962182739, 897.8411482760821, 1202.5044684430127, 1029.2274483460903]
[55.778926809290304, 38.24558622256441, 73.94626958236361, 74.82233005517422, 68.85524871487543]
0123456789

99 percent confidence intervals.
mse percent = 8.19748177986  confidence int: [4.3788417258615961, 12.016121833866499]
sqrt of mse percent = 28.6312447858  confidence int: [20.925682129530678, 34.664278203745283]
[1.1, 1.8, 1.4, 0.9, 1.6, 1.0, 1.8]
[0.22539585376179214, 0.015683070734452464, 0.4160641546840125, 0.18889006550976675, 0.15396685530997625]
[875.6540987328171, 938.89507046519, 988.5739641343262, 1167.8296179551899, 1008.2017587220876]
[70.39078219095293, 39.40636049276405, 84.85000508742306, 61.51051326531227, 68.98289752364799]
0123456789

99 percent confidence intervals.
mse percent = 4.63888998545  confidence int: [1.5719360353401386, 7.7058439355551149]
sqrt of mse 

/home/centos/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:92: RuntimeWarning: invalid value encountered in sqrt


0123456789

99 percent confidence intervals.
mse percent = 7.82262509025  confidence int: [1.2835567845060527, 14.361693395987023]
sqrt of mse percent = 27.9689561662  confidence int: [11.329416509715108, 37.896824927673059]
[1.1, 1.8, 1.4, 0.9, 1.6, 1.0, 1.8]
[0.19277590395804933, 0.0684060271041379, 0.26991207307553794, 0.1886600419298712, 0.2802459539324036]
[1114.017641967696, 903.5775962182739, 897.8411482760821, 1202.5044684430127, 1029.2274483460903]
[55.778926809290304, 38.24558622256441, 73.94626958236361, 74.82233005517422, 68.85524871487543]
0123456789

99 percent confidence intervals.
mse percent = 11.5613229463  confidence int: [4.1886867363724463, 18.933959156313229]
sqrt of mse percent = 34.0019454537  confidence int: [20.466281382734007, 43.513169450539031]
[1.1, 1.8, 1.4, 0.9, 1.6, 1.0, 1.8]
[0.22539585376179214, 0.015683070734452464, 0.4160641546840125, 0.18889006550976675, 0.15396685530997625]
[875.6540987328171, 938.89507046519, 988.5739641343262, 1167.8296179551899